### --- Day 14: Extended Polymerization ---
The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

    NNCB

    CH -> B
    HH -> N
    CB -> H
    NH -> C
    HB -> C
    HC -> B
    HN -> C
    NN -> C
    BH -> H
    NC -> B
    NB -> B
    BN -> B
    BB -> N
    BC -> B
    CC -> N
    CN -> C
The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

    Template:     NNCB
    After step 1: NCNBCHB
    After step 2: NBCCNBBBCBHCB
    After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
    After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

Your puzzle answer was 3697.

The first half of this puzzle is complete! It provides one gold star: *

### --- Part Two ---
The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [20]:
def polymerisation_input(input_file:str):
    """Parse text file input"""
    templates = []
    pairs = []
    inserts = []
    
    with open(input_file, 'r') as file:
        products = file.readline().strip()
        for line in file:
            if line.strip() == '': 
                pass
            else:
                template = line.strip().split(' -> ')
                pairs.append(template[0])
                inserts.append(template[1])
        templates = [pairs, inserts]
    return products, templates


In [100]:
# Part 1 functions - Stored as list

def insert_iteration(products:str, templates:list, iterations:int):
    for i in range(iterations):
        newline = ''
        for i, char in enumerate(products):
            if i < len(products) - 1:      
                idx = templates[0].index(products[i:i+2])
                newline += char + templates[1][idx]
            elif i == len(products) - 1:
                newline += char
        products = newline
    return newline

def find_most_common(final_product:str) -> dict:
    counts = {}
    for char in final_product:
        if char not in counts.keys():
            counts[char] = 1
        else:
            counts[char] += 1
    return sorted(counts.items(), key=lambda x:x[1])


    

In [394]:
# Workflow

# Input
products, templates = polymerisation_input('data/d14_polymerisation.txt')

# Iterate through changes
final_product = insert_iteration(products, templates, 10)

# Calculate counts
counts = find_most_common(final_product)
print(counts)

answer = counts[-1][1] - counts[0][1]
print(len(final_product))

print(f'The most of common letter is {counts[-1][0]} with {counts[-1][1]}') 
print(f'and the least common is {counts[0][0]} with {counts[0][1]}')
print(f'Answer = {answer}')

[('K', 908), ('O', 1154), ('F', 1194), ('B', 1618), ('C', 1657), ('V', 1793), ('P', 1907), ('S', 2084), ('H', 2537), ('N', 4605)]
19457
The most of common letter is N with 4605
and the least common is K with 908
Answer = 3697


In [380]:
# Part 2 functions - Stored as dict

def convert_product_format(products):
    polymer_pairs = {}

    for i, char in enumerate(products):
        if i < len(products) - 1:
            pair = char+products[i+1]
            if pair not in polymer_pairs.keys():
                polymer_pairs[pair] = 1
            else:
                polymer_pairs[pair] += 1
    return polymer_pairs

def insert_iteration_dict(polymer_pairs:dict, templates:list, iterations:int):
    print(f'Starting pairs: {polymer_pairs}')
    for i in range(iterations):
        print(f'\nIteration {i}')
        # Create copy of dict so can test on these values but edit polymer_pairs directly
        tester_pairs = dict(polymer_pairs)
        for key in list(tester_pairs):
            # check to make sure it has a count already
            if key in templates[0] and tester_pairs[key] > 0:
                # Pull out template pair and the associated letter for between
                found_idx = templates[0].index(key)
                found = templates[1][found_idx]
                key_parts = [char for char in key]
                # Create new keys
                new_keys = [key_parts[0]+found, found+key_parts[1]]
                # Insert new keys
                for k in new_keys:
                    if k in polymer_pairs.keys():
                        polymer_pairs[k] += tester_pairs[key]
                    else:
                        polymer_pairs[k] = tester_pairs[key]
                # Remove old key now it's been modified
                polymer_pairs[key] -= tester_pairs[key]
            else:
                pass
        print(f'After an iteration: {polymer_pairs}')
    return polymer_pairs

def return_counts_from_dict(polymer_pairs:dict):
    """
    Batch up polymer_pair counts, separate letters, then add up.
    
    If the number is odd, you need to add 1 then divide (start or end letter),
    If even then just divide by two.
    """
    total_counts = {}
    # Find all occurences of a letter in the pair dictionary
    for key,val in polymer_pairs.items():
        chars = [c for c in key]
        for char in chars:
            if char in total_counts.keys():
                total_counts[char] += val
            else:
                total_counts[char] = val
    # Now divide by two to account for duplicate letters in pairs (start & end)
    for key, val in total_counts.items():
        if val % 2 == 0:
            total_counts[key] = int(val / 2)
        else:
            total_counts[key] = int((val + 1) / 2)
    return total_counts

        

In [396]:
# Workflow Part 2


# Input
products, templates = polymerisation_input('data/d14_polymerisation.txt')

# Convert lists to dictionary of pairs
polymer_pairs = convert_product_format(products)
# now iterate
polymers = insert_iteration_dict(polymer_pairs, templates, 40)


Starting pairs: {'CP': 1, 'PS': 1, 'SS': 2, 'SF': 1, 'FC': 1, 'CF': 1, 'FO': 1, 'OF': 1, 'FV': 2, 'VF': 1, 'FN': 1, 'NV': 1, 'VP': 1, 'PK': 1, 'KB': 1, 'BF': 1, 'VN': 1}

Iteration 0
After an iteration: {'CP': 0, 'PS': 0, 'SS': 0, 'SF': 1, 'FC': 1, 'CF': 0, 'FO': 0, 'OF': 1, 'FV': 0, 'VF': 1, 'FN': 0, 'NV': 1, 'VP': 0, 'PK': 0, 'KB': 0, 'BF': 0, 'VN': 1, 'CB': 1, 'BP': 1, 'PN': 2, 'NS': 1, 'SO': 2, 'OS': 2, 'FF': 2, 'CC': 1, 'CO': 1, 'FB': 1, 'BO': 1, 'OH': 1, 'HF': 2, 'FP': 3, 'PV': 2, 'VV': 1, 'NP': 1, 'PO': 1, 'OK': 1, 'KV': 1, 'VB': 1, 'BH': 1, 'VH': 1, 'HN': 1}

Iteration 1
After an iteration: {'CP': 0, 'PS': 0, 'SS': 0, 'SF': 1, 'FC': 1, 'CF': 0, 'FO': 1, 'OF': 3, 'FV': 2, 'VF': 1, 'FN': 0, 'NV': 1, 'VP': 0, 'PK': 0, 'KB': 0, 'BF': 0, 'VN': 0, 'CB': 0, 'BP': 0, 'PN': 2, 'NS': 0, 'SO': 0, 'OS': 0, 'FF': 2, 'CC': 1, 'CO': 1, 'FB': 0, 'BO': 0, 'OH': 1, 'HF': 3, 'FP': 4, 'PV': 1, 'VV': 2, 'NP': 0, 'PO': 0, 'OK': 0, 'KV': 0, 'VB': 2, 'BH': 1, 'VH': 2, 'HN': 2, 'CS': 3, 'SB': 1, 'BK': 

In [397]:
# Calculate counts
counts = return_counts_from_dict(polymers)
print(f'Total sum of all elements: {sum(counts.values())}')

# Sort and answer question
sorted_counts = sorted(counts.items(), key=lambda x:x[1])
print(sorted_counts)
print(f'The answer is {sorted_counts[-1][1]} - {sorted_counts[0][1]}: ')
print(sorted_counts[-1][1] - sorted_counts[0][1])


20890720927745
Total sum of all elements: 20890720927745
[('K', 829450972300), ('O', 1222221445766), ('F', 1296815711280), ('B', 1767725849671), ('C', 1828285684896), ('V', 1928549233139), ('S', 2027995072379), ('P', 2108852305644), ('H', 2680065844213), ('N', 5200758808457)]
The answer is 5200758808457 - 829450972300: 
4371307836157
